# ByteNet Character Prediction Model & Experiment
This task uses the Hutter Prize version of  the Wikipedia dataset to test

In [1]:
!pip install datasets

In [2]:
# Define Imports
import json

import requests
import torch
import pickle
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
from datasets import load_dataset
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from transformers import AutoTokenizer


In [3]:
from datasets import load_dataset
dataset = load_dataset('enwik8')['train']

In [4]:
from bs4 import BeautifulSoup

def remove_xml_tags(example):
    soup = BeautifulSoup(example['text'], 'lxml')
    text_content = soup.get_text()
    return {'text': text_content}

def clean_html(example):
    import re
    from bs4 import BeautifulSoup
    text = example['text'] 
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text(separator=" ")
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\[\[|\]\]|\'\'', '', text)
    return {'text': text}


In [5]:
filtered_dataset = dataset.map(remove_xml_tags)
filtered_dataset = filtered_dataset.map(clean_html)

In [6]:
print(filtered_dataset[:220])

{'text': ['', '', 'Wikipedia', '', 'MediaWiki 1.6alpha', 'first-letter', '', 'Media', 'Special', '', 'Talk', 'User', 'User talk', 'Wikipedia', 'Wikipedia talk', 'Image', 'Image talk', 'MediaWiki', 'MediaWiki talk', 'Template', 'Template talk', 'Help', 'Help talk', 'Category', 'Category talk', 'Portal', 'Portal talk', '', '', '', 'AaA', '1', '', '32899315', '2005-12-27T18:46:47Z', '', 'Jsmethers', '614213', '', '#REDIRECT AAA', '', '', '', 'AlgeriA', '5', '', '18063769', '2005-07-03T11:13:13Z', '', 'Docu', '8029', '', '', 'adding cur_id=5: {{R from CamelCase}}', '#REDIRECT Algeria{{R from CamelCase}}', '', '', '', 'AmericanSamoa', '6', '', '18063795', '2005-07-03T11:14:17Z', '', 'Docu', '8029', '', '', 'adding to cur_id=6  {{R from CamelCase}}', '#REDIRECT American Samoa{{R from CamelCase}}', '', '', '', 'AppliedEthics', '8', '', '15898943', '2002-02-25T15:43:11Z', '', 'Conversion script', '', '', 'Automated conversion', '#REDIRECT Applied ethics', '', '', '', '', 'AccessibleComputing',

In [7]:
def join_to_length(data, target_length):
    joined_data = []
    temp_str = ''
    for entry in data:
        temp_str += entry
        if len(temp_str) >= target_length:
            joined_data.append(temp_str[:target_length])
            temp_str = temp_str[target_length:]
    if temp_str:  # add remaining string if it's not empty
        joined_data.append(temp_str)
    return joined_data

In [8]:
train_dataset = join_to_length(filtered_dataset['text'],500)

In [9]:
print(train_dataset[:150])

['WikipediaMediaWiki 1.6alphafirst-letterMediaSpecialTalkUserUser talkWikipediaWikipedia talkImageImage talkMediaWikiMediaWiki talkTemplateTemplate talkHelpHelp talkCategoryCategory talkPortalPortal talkAaA1328993152005-12-27T18:46:47ZJsmethers614213#REDIRECT AAAAlgeriA5180637692005-07-03T11:13:13ZDocu8029adding cur_id=5: {{R from CamelCase}}#REDIRECT Algeria{{R from CamelCase}}AmericanSamoa6180637952005-07-03T11:14:17ZDocu8029adding to cur_id=6  {{R from CamelCase}}#REDIRECT American Samoa{{R fro', "m CamelCase}}AppliedEthics8158989432002-02-25T15:43:11ZConversion scriptAutomated conversion#REDIRECT Applied ethicsAccessibleComputing10158989452003-04-25T22:18:38ZAms807543Fixing redirect#REDIRECT Accessible_computingAdA11158989462002-09-22T16:02:58ZAndre Engels300#REDIRECT Ada programming languageAnarchism12421368312006-03-04T01:41:25ZCJames745832382/* Anarchist Communism */  too many brackets{{Anarchism}}'Anarchism' originated as a term of abuse first used against early working class r

In [10]:
print(len(train_dataset))

170085


In [11]:
train_split = int(0.9 * len(train_dataset))
validation_split = int(0.95 * len(train_dataset))
test_split = len(train_dataset)

In [12]:
# Takes too long for full dataset
train_data = train_dataset[:int(0.4*train_split)]
validation_data = train_dataset[train_split:validation_split]
test_data = train_dataset[validation_split:test_split]

In [13]:
print(len(train_data),len(validation_data),len(test_data))

61230 8504 8505


In [14]:
tokenizer = AutoTokenizer.from_pretrained("google/byt5-small")

In [15]:
def tokenize_item(item):
    src = tokenizer(item[:100], max_length=100, padding="max_length", truncation=True, return_tensors="pt")
    trgt = tokenizer(item[100:], max_length=400, padding="max_length", truncation=True, return_tensors="pt")
    return src['input_ids'],trgt['input_ids']

In [16]:
tokenize_item(train_data[1])

(tensor([[112,  35,  70, 100, 112, 104, 111,  70, 100, 118, 104, 128, 128,  68,
          115, 115, 111, 108, 104, 103,  72, 119, 107, 108, 102, 118,  59,  52,
           56,  59,  60,  59,  60,  55,  54,  53,  51,  51,  53,  48,  51,  53,
           48,  53,  56,  87,  52,  56,  61,  55,  54,  61,  52,  52,  93,  70,
          114, 113, 121, 104, 117, 118, 108, 114, 113,  35, 118, 102, 117, 108,
          115, 119,  68, 120, 119, 114, 112, 100, 119, 104, 103,  35, 102, 114,
          113, 121, 104, 117, 118, 108, 114, 113,  38,  85,  72,  71,  76,  85,
           72,   1]]),
 tensor([[ 87,  35,  68, 115, 115, 111, 108, 104, 103,  35, 104, 119, 107, 108,
          102, 118,  68, 102, 102, 104, 118, 118, 108, 101, 111, 104,  70, 114,
          112, 115, 120, 119, 108, 113, 106,  52,  51,  52,  56,  59,  60,  59,
           60,  55,  56,  53,  51,  51,  54,  48,  51,  55,  48,  53,  56,  87,
           53,  53,  61,  52,  59,  61,  54,  59,  93,  68, 112, 118,  59,  51,
           58,  5

In [17]:
# Imports for ByteNet
import torch
from torch.utils.data import Dataset
from torch import nn
from torch.nn import init
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm

In [18]:
class DynamicLayerNorm(nn.Module):
    def __init__(self):
        super(DynamicLayerNorm, self).__init__()
        self.norm = None

    def forward(self, x):
        if self.norm is None or self.norm.normalized_shape[0] != x.shape[2]:
            self.norm = nn.LayerNorm(x.shape[2]).to(x.device)
        return self.norm(x)

In [19]:
class MultiplicativeUnit(nn.Module):
    def __init__(self, d, dilation, k, masked = True):
        super(MultiplicativeUnit, self).__init__()
        self.receptive_field = (k-1)*dilation
        self.masked = masked
        self.sig_conv_1 = nn.Conv1d(d, d, k, dilation = dilation)
        self.sig_conv_2 = nn.Conv1d(d, d, k, dilation = dilation)
        self.sig_conv_3 = nn.Conv1d(d, d, k, dilation = dilation)
        self.tanh_conv_1 = nn.Conv1d(d, d, k, dilation = dilation)

        # 100 input tokens
        self.layer_norm1 = DynamicLayerNorm()
        self.layer_norm2 = DynamicLayerNorm()
        self.layer_norm3 = DynamicLayerNorm()
        self.layer_norm4 = DynamicLayerNorm()
        
    def forward(self,x):
        residual = x
        # Multiplicative Unit Block
        if self.receptive_field > 0 and self.masked:
            x = F.pad(x, (self.receptive_field, 0))
        sig_1 = torch.sigmoid(self.layer_norm1(self.sig_conv_1(x)))
        sig_2 = torch.sigmoid(self.layer_norm2(self.sig_conv_2(x)))
        sig_3 = torch.sigmoid(self.layer_norm3(self.sig_conv_3(x)))
        tanh_1 = torch.tanh(self.layer_norm4(self.tanh_conv_1(x)))
        sig_tanh = sig_2 * tanh_1
        residual_sig = sig_3 * residual
        mu_out = sig_1 * torch.tanh(sig_tanh+residual_sig)
        # Multiplicative unit block end
        # Add back the residual
        mu_out = mu_out * residual
        return mu_out


In [20]:
class ResidualBlockMu(nn.Module):
    """
    Implementation of residual Layer for Bytenet machine translation task.

    :param d: The number of input features.
    :param dilation: The initial dilation rate for the convolution layers.
    """

    def __init__(self, d, dilation, k):
        super(ResidualBlockMu, self).__init__()
        self.layer_norm_in = DynamicLayerNorm()
        self.conv_in_1 = nn.Conv1d(2*d,d,1)
        self.layer_norm_in_2 = DynamicLayerNorm()
        # Multiplicative block
        # d -> d
        self.masked_mu = MultiplicativeUnit(d,dilation,k)
        self.mu = MultiplicativeUnit(d,1,1, masked=False)
        self.conv_out = nn.Conv1d(d,2*d,1)

    def forward(self, x):
        residual = x
        x = self.layer_norm_in(x)
        x = torch.nn.ReLU()(x)
        x = self.conv_in_1(x)
        x = self.layer_norm_in_2(x)
        x = torch.nn.ReLU()(x)
        x = self.masked_mu(x)
        x = self.mu(x)
        x = self.conv_out(x)
        x = x + residual
        return x


In [21]:
class HutterDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        src, trgt = tokenize_item(item)
        return src, trgt


In [22]:
train_dataset = HutterDataset(train_data)
validation_dataset = HutterDataset(validation_data)
test_dataset = HutterDataset(test_data)

In [23]:
batch_size = 8
shuffle = True

In [24]:
train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=shuffle)
validation_loader = DataLoader(validation_dataset,batch_size=batch_size,shuffle=shuffle)
test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=shuffle)

In [25]:
class ByteNetDecoder(nn.Module):
    def __init__(self, vocab_size, embed_size=1024, d=512, n_sets = 4, set_size = 5, masked_kernel_size=3, max_dilation_rate=16):
        super(ByteNetDecoder, self).__init__()

        self.embedding = nn.Embedding(vocab_size,embed_size)
        self.layers = nn.Sequential()

        for _ in range(n_sets):
            dilation_rate = 1
            for _ in range(set_size):
                self.layers.append(ResidualBlockMu(d,dilation_rate if dilation_rate <= max_dilation_rate else max_dilation_rate, masked_kernel_size))
                dilation_rate = dilation_rate * 2
        self.layers.append(nn.Conv1d(d * 2, d, 1))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.Conv1d(d, vocab_size, 1))
        self.layers.append(nn.Dropout(p=0.1))
    def forward(self,x):
        embed_x = self.embedding(x).permute(0, 2, 1)
        x = self.layers(embed_x)
        return x

In [26]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
decoder = ByteNetDecoder(len(tokenizer.get_vocab())).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(decoder.parameters(),lr = 0.0003, weight_decay=0.0001)

In [27]:
num_epochs = 2

In [28]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

2024-06-26 15:35:07.423308: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-26 15:35:07.470022: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-26 15:35:07.470088: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-26 15:35:07.472729: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-26 15:35:07.485689: I tensorflow/core/platform/cpu_feature_guar

In [29]:
def unfold(inp, decoder):
    x = inp
    out = None
    # 100 -> 200 -> 400 output characters
    for i in range(4):
        # 100 as input -> 200 as input -> 400 as input
        out = decoder(x)
        next_gen = torch.argmax(out,1)
        # 200 characters  ( 100 predicted, 100 given as input) -> (300 predicted, 100 input)
        x =  torch.cat((x,next_gen), dim = 1)
    return out[:, :, out.size(2)//2:]

In [ ]:
for epoch in range(num_epochs):
    for i, (ip,trgt) in tqdm(enumerate(train_loader), total=len(train_loader)):
        decoder.train()
        ip, trgt = ip.squeeze(1).to(device), trgt.squeeze(1).to(device)
        out = unfold(ip,decoder)
        loss = criterion(out,trgt)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        step = (i + 1)/len(train_loader)
        if i % 25 == 0:
            writer.add_scalar("Loss/train",loss,step)
            tqdm.write(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item()}')
        if i % 2000 == 0 and i > 0:
            decoder.eval()
            total_val_loss = 0
            with torch.no_grad():
                for i, (ip, trgt) in enumerate(validation_loader):
                    ip, trgt = ip.squeeze(1).to(device), trgt.squeeze(1).to(device)
                    out = unfold(ip,decoder)
                    val_loss = criterion(out, trgt)
                    total_val_loss += val_loss.item()
        
            avg_val_loss = total_val_loss / len(validation_loader)
            writer.add_scalar("Loss/val", avg_val_loss, step)
            tqdm.write(f'Epoch [{epoch + 1}/{num_epochs}], , Step [{i + 1}/{len(train_loader)}], Val Loss: {avg_val_loss}')

  0%|          | 1/7654 [00:02<6:11:05,  2.91s/it]

Epoch [1/2], Step [1/7654], Loss: 5.932135105133057


  0%|          | 26/7654 [00:50<4:20:50,  2.05s/it]

Epoch [1/2], Step [26/7654], Loss: 3.7533555030822754


  1%|          | 51/7654 [01:39<4:20:57,  2.06s/it]

...

 55%|█████▍    | 4176/7654 [2:47:31<1:59:33,  2.06s/it]

Epoch [1/2], Step [4176/7654], Loss: 3.5543596744537354


 55%|█████▍    | 4201/7654 [2:48:19<1:59:09,  2.07s/it]

Epoch [1/2], Step [4201/7654], Loss: 3.533764600753784


 55%|█████▌    | 4218/7654 [2:48:52<1:50:07,  1.92s/it]

In [ ]:
!tensorboard --logdir=runs

In [ ]:
torch.save(decoder.state_dict(), 'model_state.pth')

In [ ]:
writer.close()

In [ ]:
decoder = ByteNetDecoder(len(tokenizer.get_vocab())).to(device)
decoder.load_state_dict(torch.load('model_state.pth'))

In [ ]:
def predict(inp, model,  tokenizer):
    model.eval()
    with torch.inference_mode():
        out = model(inp)
    print(out.shape)
    out = torch.argmax(out.squeeze(0), dim=0)

    out = tokenizer.decode(out.tolist())
    out = out.replace("<pad>", "")
    print(f"Predicted text:  {out}")

In [ ]:
batch = next(iter(validation_loader))
batch =  next(iter(train_loader))

In [ ]:
src = batch[0][0][0]
trgt = batch[1][0][0]
i =  tokenizer.decode(src.tolist())
i = i.replace("<pad>", "")
print(f"Input : {i}")
print(f"Ground  Truth : {tokenizer.decode(trgt.tolist())}")


In [ ]:
predict(batch[0][0].to(device),decoder,tokenizer)